In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)
#FIX ME Add in Grazioso Salvare's logo
image_filename = 'grazzios.png'  # using your uploaded image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    # Add logo and your name as identifier
    html.Div([
        html.A([
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                    style={'height': '100px', 'width': 'auto'})
        ], href='https://www.snhu.edu'),
        html.H3("Created by [Dennis Selfinger]", style={'margin-left': '20px'})
    ], style={'display': 'flex', 'align-items': 'center', 'justify-content': 'center'}),
    
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
        
        #FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset',
            labelStyle={'display': 'block'}
        )
    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         #FIXME: Set up the features to make this interactive data table to make it user-friendly 
                         #for your client
                         page_current=0,
                         page_size=10,
                         page_action='native',
                         sort_action='native',
                         sort_mode='single',
                         style_table={'overflowX': 'auto'},
                         style_cell={
                             'height': 'auto',
                             'minWidth': '180px', 'width': '180px', 'maxWidth': '180px',
                             'whiteSpace': 'normal'
                         }
                        ),
    html.Br(),
    html.Hr(),
    #This sets up the dashboard so that the  chart and the geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])
'''
#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'grazzios.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.

    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 

                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])
'''
#############################################
# Interaction Between Components / Controller
#############################################



    
#############################################
# Interaction Between Components / Controller
#############################################

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(Output('datatable-id', 'data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    # Filter interactive data table with MongoDB queries
    if filter_type == 'water':
        # Water rescue: Labrador, Chesapeake, Newfoundland + Female + 26-156 weeks
        query = {"$and": [
            {"breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}},
            {"sex_upon_outcome": "Intact Female"},
            {"age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
        ]}
        results = db.read(query)
    elif filter_type == 'mountain':
        # Mountain rescue: German Shepherd, etc. + Male + 26-156 weeks
        query = {"$and": [
            {"breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", 
                               "Siberian Husky", "Rottweiler"]}},
            {"sex_upon_outcome": "Intact Male"},
            {"age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
        ]}
        results = db.read(query)
    elif filter_type == 'disaster':
        # Disaster: Doberman, German Shepherd, etc. + Male + 20-300 weeks
        query = {"$and": [
            {"breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", 
                              "Bloodhound", "Rottweiler"]}},
            {"sex_upon_outcome": "Intact Male"},
            {"age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}}
        ]}
        results = db.read(query)
    else:
        # Reset - return all
        results = db.read({})
        
    df_filtered = pd.DataFrame.from_records(results)
    
    # Remove _id column to prevent crashes
    if '_id' in df_filtered.columns:
        df_filtered.drop(columns=['_id'], inplace=True)
        
    return df_filtered.to_dict('records')


# Display the breeds of animal based on quantity represented in the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        # Return empty div if no data
        return [html.Div("No data available")]
        
    dff = pd.DataFrame.from_dict(viewData)
    
    # Create pie chart based on breed distribution
    return [
        dcc.Graph(            
            figure=px.pie(dff, names='breed', title='Breed Distribution')
        )    
    ]
    

# This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    # Check if we have data
    if viewData is None or len(viewData) == 0:
        return [html.Div("No data to display on map")]
    
    try:
        dff = pd.DataFrame.from_dict(viewData)
        
        # Handle missing location data
        if 'location_lat' not in dff.columns or 'location_long' not in dff.columns:
            return [html.Div("Missing location data for map display")]
            
        # Check if any row is selected
        if index is None or len(index) == 0:
            row = 0
        else: 
            row = index[0]
            
        # Make sure row index is valid
        if row >= len(dff):
            row = 0
            
        # Check if location data exists for this row
        if pd.isna(dff.iloc[row]['location_lat']) or pd.isna(dff.iloc[row]['location_long']):
            return [html.Div("Selected animal has no location data")]
        
        # Austin TX is at [30.75,-97.48]
        return [
            dl.Map(style={'width': '100%', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                # Marker with tool tip and popup
                dl.Marker(position=[dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']], children=[
                    dl.Tooltip(dff.iloc[row]['breed']),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(dff.iloc[row]['name'] if 'name' in dff.columns else "No name available")
                    ])
                ])
            ])
        ]
    except Exception as e:
        # Return error message if anything goes wrong
        return [html.Div(f"Error displaying map: {str(e)}")]

app.run_server(debug=True)

Dash app running on http://127.0.0.1:13741/
